In [1]:
import pandas as pd
from datetime import timedelta

In [2]:
# データのサンプル
df = pd.DataFrame({
    'user': ['A', 'A', 'B', 'C', 'B', 'A', 'B', 'B'],
    'date': ['2022-05-05', '2022-05-05', '2022-05-05', '2022-04-15', '2022-04-15', '2022-03-22', '2022-03-22', '2022-03-22'],
    'item': ['a', 'b', 'c', 'd', 'a', 'c', 'd', 'e'],
})

# あるいは、csvから読み込む
# df = pd.read_csv('df_sample.csv')


In [3]:
df

,user,date,item
0,A,2022-05-05,a
1,A,2022-05-05,b
2,B,2022-05-05,c
3,C,2022-04-15,d
4,B,2022-04-15,a
5,A,2022-03-22,c
6,B,2022-03-22,d
7,B,2022-03-22,e


In [4]:
# ユーザ名と日付のカラム
user_col_name = 'user'
date_col_name = 'date'

# 日付のフォーマット
date_format = '%Y-%m-%d'

# 日付の差の閾値
days_threshold = 30

In [5]:
df.dtypes

user    object
date    object
item    object
dtype: object

In [6]:
# 日付を文字列からdatetime64[ns]に変換
df[date_col_name] = pd.to_datetime(df[date_col_name], format=date_format)

In [7]:
df

,user,date,item
0,A,2022-05-05,a
1,A,2022-05-05,b
2,B,2022-05-05,c
3,C,2022-04-15,d
4,B,2022-04-15,a
5,A,2022-03-22,c
6,B,2022-03-22,d
7,B,2022-03-22,e


In [8]:
df.dtypes

user            object
date    datetime64[ns]
item            object
dtype: object

In [9]:
# 必要なカラムだけ選択し、日付の重複を削除
df = df[[user_col_name, date_col_name]].drop_duplicates()

In [10]:
df

,user,date
0,A,2022-05-05
2,B,2022-05-05
3,C,2022-04-15
4,B,2022-04-15
5,A,2022-03-22
6,B,2022-03-22


In [11]:
# ユーザ名順、新しい順に、並び替え
df = df.sort_values(by=[user_col_name, date_col_name], ascending=[True, False])

In [12]:
df

,user,date
0,A,2022-05-05
5,A,2022-03-22
2,B,2022-05-05
4,B,2022-04-15
6,B,2022-03-22
3,C,2022-04-15


In [13]:
# 各ユーザの最新
df.groupby(user_col_name)[date_col_name].nth(0)

user
A   2022-05-05
B   2022-05-05
C   2022-04-15
Name: date, dtype: datetime64[ns]

In [14]:
# 各ユーザの最新の1つ前
df.groupby(user_col_name)[date_col_name].nth(1)

user
A   2022-03-22
B   2022-04-15
Name: date, dtype: datetime64[ns]

In [15]:
# 2日以上購入しているユーザの、最新とその前のデータ
dfg = df.groupby(user_col_name)[date_col_name]
df_latest = pd.DataFrame(dfg.nth(0)).reset_index()
df_second_latest = pd.DataFrame(dfg.nth(1)).reset_index()
df2 = df_latest.merge(df_second_latest, on=user_col_name, how='inner', suffixes=('_latest', '_second_latest'))

In [16]:
df2

,user,date_latest,date_second_latest
0,A,2022-05-05,2022-03-22
1,B,2022-05-05,2022-04-15


In [17]:
# 日付の差分を計算
df2['date_diff'] = df2[f'{date_col_name}_latest'] - df2[f'{date_col_name}_second_latest']

In [18]:
df2


,user,date_latest,date_second_latest,date_diff
0,A,2022-05-05,2022-03-22,44 days
1,B,2022-05-05,2022-04-15,20 days


In [19]:
# 閾値以上のデータを取得
dm = df2[df2['date_diff'] >= timedelta(days=days_threshold)]

In [20]:
dm

,user,date_latest,date_second_latest,date_diff
0,A,2022-05-05,2022-03-22,44 days


In [21]:
# dmを保存
dm.to_csv('dm.csv', index=False)

In [22]:
# ユーザ名だけ保存
dm[[user_col_name]].to_csv('user.csv', index=False)